In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
train_dir = '/kaggle/input/original-smote-image-processing/split/train/'
test_dir = '/kaggle/input/original-smote-image-processing/split/test/'

In [ ]:
batch_size=128
learning_rate=0.001
epochs=50
optimizer=(tf.keras.optimizers.Adam(learning_rate=learning_rate))
dropout=0.2

In [ ]:
train_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

val_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

In [ ]:
train_images = np.concatenate(list(train_data.map(lambda x, y:x)))
train_labels = np.concatenate(list(train_data.map(lambda x, y:y)))

val_images = np.concatenate(list(val_data.map(lambda x, y:x)))
val_labels = np.concatenate(list(val_data.map(lambda x, y:y)))

inputs = np.concatenate((train_images, val_images), axis=0)
targets = np.concatenate((train_labels, val_labels), axis=0)

kfold = KFold(n_splits=3, shuffle=True)


In [ ]:
fold=1
for train, test in kfold.split(inputs, targets):
    optimizer=(tf.keras.optimizers.Adam(learning_rate=0.001))
    model = Sequential()
    pretrained_model = MobileNetV3Small(
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='max',
        weights='imagenet'
    )
    pretrained_model.trainable = False
    pretrained_model = Model(
        inputs=pretrained_model.inputs,
        outputs=pretrained_model.layers[-2].output
    )
    model.add(pretrained_model)
    model.add(BatchNormalization())
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(18, activation='softmax'))

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(inputs[test], targets[test]),
              callbacks=[tensorboard_callback])

    test_loss, test_accuracy = model.evaluate(test_data, verbose=0)
    print(f"Test Accuracy for Fold {fold}: {test_accuracy}")
    fold+=1